In [1]:
from main import GWAnalyzer
import torch

In [ ]:
batch_size: int = 384
ifos: int = 2
sample_rate: int = 2048
kernel_length = 1.5
X = torch.randn(batch_size, ifos, int(sample_rate * kernel_length), device="cuda")

In [13]:
detector1 = X[:, 0, :]
detector2 = X[:, 1, :]
gwana = GWAnalyzer(detector1.cpu().numpy())
gwana.obtain_topological_features(True, True)
features1 = torch.tensor(gwana.topological_features)
gwana = GWAnalyzer(detector2.cpu().numpy())
gwana.obtain_topological_features(True, True)
features2 = torch.tensor(gwana.topological_features)

start processing spectrograms


0it [00:00, ?it/s]

Processing chunk of shape: (384, 1417, 3)
Chunk Elapsed Time: 2.7869298458099365
start processing point cloud features


0it [00:00, ?it/s]

Processing chunk of shape: (384, 3072)
Chunk Elapsed Time: 75.95832657814026
Shape of the final features is (384, 36)
start processing spectrograms


0it [00:00, ?it/s]

Processing chunk of shape: (384, 1424, 3)
Chunk Elapsed Time: 2.9170761108398438
start processing point cloud features


0it [00:00, ?it/s]

Processing chunk of shape: (384, 3072)
Chunk Elapsed Time: 76.56115245819092
Shape of the final features is (384, 36)


In [14]:
features = torch.stack([features1, features2], dim=1)
features.shape

torch.Size([384, 2, 36])